<a href="https://colab.research.google.com/github/Hari1331/Hari_DS/blob/master/Tf_Load_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install Tensorflow beta version

Install tensorflow bets version

In [1]:
!pip install tensorflow==2.0.0-beta1

Import Numpy and tensorflow library

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
import functools

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

Get the Train and test date from Site storage.googleapis.com/tf.dataset

In [3]:
TRAIN_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/train.csv"
TEST_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/eval.csv"

train_file_path = tf.keras.utils.get_file("train.csv", TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file("eval.csv", TEST_DATA_URL)

16384/13049 [=====================================] - 0s 0us/step


**numpy.set_printoptions**. These options determine the way floating point numbers, arrays and other NumPy objects are displayed.

In [0]:
np.set_printoptions(precision=3, suppress=True)

Using Head function ,Read top 10 rows in the csv file path

In [5]:
!head {train_file_path}

survived,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,male,22.0,1,0,7.25,Third,unknown,Southampton,n
1,female,38.0,1,0,71.2833,First,C,Cherbourg,n
1,female,26.0,0,0,7.925,Third,unknown,Southampton,y
1,female,35.0,1,0,53.1,First,C,Southampton,n
0,male,28.0,0,0,8.4583,Third,unknown,Queenstown,y
0,male,2.0,3,1,21.075,Third,unknown,Southampton,n
1,female,27.0,0,2,11.1333,Third,unknown,Southampton,n
1,female,14.0,1,0,30.0708,Second,unknown,Cherbourg,n
1,female,4.0,1,1,16.7,Third,G,Southampton,n


Each column of the CSV file will have a column name. The dataset's constructor automatically recognizes these column names. If the first line of the file you are using does not contain a column name, you need to pass the column name to the column_names parameter of the make_csv_dataset function via the list of strings.

CSV_COLUMNS = ['survived', 'sex', 'age', 'n_siblings_spouses', 'parch', 'fare', 'class', 'deck', 'embark_town', 'alone']

dataset = tf.data.experimental.make_csv_dataset(
     ...,
     column_names=CSV_COLUMNS,
     ...)

This example uses all the columns. If you need to ignore certain columns in the dataset, create a list containing the columns you need to use and pass it to the constructor's (optional) parameter select_columns.

dataset = tf.data.experimental.make_csv_dataset(
  ...,
  select_columns = columns_to_use, 
  ...)

The columns that contain the values that the model needs to predict are explicitly specified. .

In [0]:
LABEL_COLUMN = 'survived'
LABELS = [0, 1]

Now read the CSV data from the file and create a dataset
For complete documentation, see tf.data.experimental.make_csv_dataset

In [0]:
def get_dataset(file_path):
  dataset = tf.data.experimental.make_csv_dataset(
      file_path,
      batch_size=12, 
      label_name=LABEL_COLUMN,
      na_value="?",
      num_epochs=1,
      ignore_errors=True)
  return dataset

raw_train_data = get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path)

In [10]:
examples, labels = next(iter(raw_train_data)) 
print("EXAMPLES: \n", examples, "\n")
print("LABELS: \n", labels)

EXAMPLES: 
 OrderedDict([('sex', <tf.Tensor: id=326, shape=(12,), dtype=string, numpy=
array([b'male', b'male', b'female', b'female', b'female', b'male',
       b'female', b'male', b'male', b'male', b'female', b'male'],
      dtype=object)>), ('age', <tf.Tensor: id=318, shape=(12,), dtype=float32, numpy=
array([36., 28., 35., 32., 28., 35., 27., 17., 44., 29., 39., 71.],
      dtype=float32)>), ('n_siblings_spouses', <tf.Tensor: id=324, shape=(12,), dtype=int32, numpy=array([1, 0, 0, 0, 1, 0, 1, 0, 2, 1, 1, 0], dtype=int32)>), ('parch', <tf.Tensor: id=325, shape=(12,), dtype=int32, numpy=array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], dtype=int32)>), ('fare', <tf.Tensor: id=323, shape=(12,), dtype=float32, numpy=
array([ 15.55 ,   7.775, 512.329,  13.   ,  24.   , 512.329,  13.858,
         8.663,  90.   ,   7.046,  83.158,  49.504], dtype=float32)>), ('class', <tf.Tensor: id=320, shape=(12,), dtype=string, numpy=
array([b'Third', b'Third', b'First', b'Second', b'Second', b'First',
       

**Classification data**
Some of the columns in the CSV data are classified columns. That is, these columns can only take values in a limited set.
Create a *tf.feature_column.indicator_column* collection using the *tf.feature_column API*, with each *tf.feature_column.indicator_column* corresponding to a **categorized column**.

In [0]:
CATEGORIES = {
    'sex': ['male', 'female'],
    'class' : ['First', 'Second', 'Third'],
    'deck' : ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    'embark_town' : ['Cherbourg', 'Southhampton', 'Queenstown'],
    'alone' : ['y', 'n']
}

In [0]:
categorical_columns = []
for feature, vocab in CATEGORIES.items():
  cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
  categorical_columns.append(tf.feature_column.indicator_column(cat_col))

In [13]:
categorical_columns

[IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='class', vocabulary_list=('First', 'Second', 'Third'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='deck', vocabulary_list=('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='embark_town', vocabulary_list=('Cherbourg', 'Southhampton', 'Queenstown'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='alone', vocabulary_list=('y', 'n'), dtype=tf.string, default_value=-1, num_oov_buckets=0))]

**Continuous data**

**Continuous data needs to be standardized**
Write a function to normalize these values and then transform those values into 2D tensors.

In [0]:
def process_continuous_data(mean, data):
  data = tf.cast(data, tf.float32) * 1/(2*mean)
  return tf.reshape(data, [-1, 1])

Now create a collection of numeric columns. The tf.feature_columns.numeric_column API uses the normalizer_fn parameter. Use functools.partial when passing arguments, and functools.partial consists of functions that are normalized using the mean of each column.

In [0]:
MEANS = {
    'age' : 29.631308,
    'n_siblings_spouses' : 0.545455,
    'parch' : 0.379585,
    'fare' : 34.385399
}

numerical_columns = []

for feature in MEANS.keys():
  num_col = tf.feature_column.numeric_column(feature, normalizer_fn=functools.partial(process_continuous_data, MEANS[feature]))
  numerical_columns.append(num_col)

In [16]:
numerical_columns

[NumericColumn(key='age', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function process_continuous_data at 0x7f473d4db2f0>, 29.631308)),
 NumericColumn(key='n_siblings_spouses', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function process_continuous_data at 0x7f473d4db2f0>, 0.545455)),
 NumericColumn(key='parch', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function process_continuous_data at 0x7f473d4db2f0>, 0.379585)),
 NumericColumn(key='fare', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function process_continuous_data at 0x7f473d4db2f0>, 34.385399))]

**Create a pre-processing layer**

Add the set of these two feature columns and pass it to tf.keras.layers.DenseFeatures to create an input layer for preprocessing.

In [0]:
preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns+numerical_columns)

**Building a model**

Build tf.keras.Sequential from preprocessing_layer.

In [0]:
model = tf.keras.Sequential([
  preprocessing_layer,
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid'),
])

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])

**Training, assessment and prediction**

It is now possible to instantiate and train the model

In [0]:
train_data = raw_train_data.shuffle(500)
test_data = raw_test_data

In [21]:
model.fit(train_data, epochs=20)

Epoch 1/20


W0729 15:09:53.149945 139945016321920 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/feature_column/feature_column_v2.py:2655: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0729 15:09:53.169103 139945016321920 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/feature_column/feature_column_v2.py:4215: IndicatorColumn._variable_shape (from tensorflow.python.feature_column.feature_column_v2) is deprecated and will be removed in a future version.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
W0729 15:09:53.170221 139945016321920 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/feature_column/feature_column_v2.py:4270: VocabularyListCateg

53/53 [==============================] - 2s 35ms/step - loss: 0.5214 - accuracy: 0.7435
Epoch 2/20
53/53 [==============================] - 0s 3ms/step - loss: 0.4276 - accuracy: 0.8228
Epoch 3/20
53/53 [==============================] - 0s 3ms/step - loss: 0.4113 - accuracy: 0.8297
Epoch 4/20
53/53 [==============================] - 0s 4ms/step - loss: 0.4010 - accuracy: 0.8523
Epoch 5/20
53/53 [==============================] - 0s 3ms/step - loss: 0.3927 - accuracy: 0.8573
Epoch 6/20
53/53 [==============================] - 0s 3ms/step - loss: 0.3862 - accuracy: 0.8574
Epoch 7/20
53/53 [==============================] - 0s 4ms/step - loss: 0.3808 - accuracy: 0.8576
Epoch 8/20
53/53 [==============================] - 0s 4ms/step - loss: 0.3756 - accuracy: 0.8588
Epoch 9/20
53/53 [==============================] - 0s 3ms/step - loss: 0.3712 - accuracy: 0.8609
Epoch 10/20
53/53 [==============================] - 0s 4ms/step - loss: 0.3670 - accuracy: 0.8608
Epoch 11/20
53/53 [==========

When the model training is complete, you can check the accuracy on the test set test_data.

In [22]:
test_loss, test_accuracy = model.evaluate(test_data)

print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))

     22/Unknown - 1s 26ms/step - loss: 0.4386 - accuracy: 0.8106

Test Loss 0.4386001236059449, Test Accuracy 0.810606062412262


Infer a label for one or more batches using tf.keras.Model.predict.

In [23]:
predictions = model.predict(test_data)
for prediction, survived in zip(predictions[:10], list(test_data)[0][1][:10]):
  print("Predicted survival: {:.2%}".format(prediction[0]),
        " | Actual outcome: ",
        ("SURVIVED" if bool(survived) else "DIED"))

Predicted survival: 86.33%  | Actual outcome:  DIED
Predicted survival: 99.82%  | Actual outcome:  SURVIVED
Predicted survival: 12.17%  | Actual outcome:  DIED
Predicted survival: 15.31%  | Actual outcome:  DIED
Predicted survival: 30.53%  | Actual outcome:  DIED
Predicted survival: 12.43%  | Actual outcome:  DIED
Predicted survival: 27.71%  | Actual outcome:  DIED
Predicted survival: 4.65%  | Actual outcome:  DIED
Predicted survival: 75.25%  | Actual outcome:  SURVIVED
Predicted survival: 89.47%  | Actual outcome:  DIED
